In [1]:
from tqdm.auto import tqdm 
import pandas as pd
import numpy as np
import xarray as xr
import netCDF4 as nf
from netCDF4 import Dataset
%matplotlib inline
import glob
import cartopy.crs as ccrs
import matplotlib.pyplot as plt

In [2]:
datapath="/work/FAC/FGSE/IDYST/tbeucler/default/saranya/Data/ECMWF/ERA5_25kmx3hr/"
path="/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/"
output="/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/outputs/"
target="/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/outputs/targets/"

In [110]:
tracklist = sorted(glob.glob(path+'tracks_wp/*_2011*'))

In [111]:
tracklist

['/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/tracks_wp/nwp_20110519.csv',
 '/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/tracks_wp/nwp_20110616.csv',
 '/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/tracks_wp/nwp_20110926.csv',
 '/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/tracks_wp/nwp_20111211.csv']

In [112]:
dm1=xr.open_mfdataset([datapath+'/vor/vor_2011.nc',datapath+'/rhum/rhum_2011.nc'])

In [96]:
def largearea_withpres(dataset=None,invar=None,indices=None,tc_irad=None):
    ds = xr.Dataset(
    data_vars=dict(variable=(["time","plev","lat","lon"], invar)),#mysvar[0])),
    coords=dict(lat=(["lat"], dataset.lat.data),lon=(["lon"], dataset.lon.data),time=(["time"], np.linspace(0,len(indices)-1,len(indices))),
               plev=(["plev"],dataset.plev.data)),
    attrs=dict(description="coords with matrices"),)
    
    LATN,LATS,LONE,LONW = tc_irad[0,:]
    testsmall = ds['variable'][0,0,:,:].sel(lat=slice(LATS,LATN),lon=slice(LONE,LONW))
    if testsmall.shape[0]<testsmall.shape[1]:
        rgspt = int(testsmall.shape[0])
    else:
        rgspt = int(testsmall.shape[1])
    var_out=np.zeros((len(indices),len(dataset.plev.data),rgspt,rgspt))
    del testsmall
    
    for it in range(len(indices)):
        latn, lats, lone, lonw = tc_irad[it,:]
        for ip in range(len(dm1.plev.data)):
            try:
                var_out[it,ip,:,:]=ds['variable'][it,ip,:,:].sel(lat=slice(lats,latn),lon=slice(lone,lonw))
            except:
                var_out[it,ip,:,:]=ds['variable'][it,ip,:,:].sel(lat=slice(lats,latn),lon=slice(lone,lonw))[0:rgspt,0:rgspt]
    return var_out  

In [97]:
def output_indices(TCtrack=None,ERA5date=None,ERA5hour=None):
    allindices = []
    for timeidx in range(len(TCtrack)):#len(track['time'])):
        datetrack,hourtrack = TCtrack['time'][timeidx].split(':')[0],TCtrack['time'][timeidx].split(':')[1][0:2]
        ####################################################################################################
        # Find the indices in ERA5 data with the same date as track
        ####################################################################################################
        dateind = []
        for ind,obj in enumerate(ERA5date):
            if obj==datetrack:
                dateind.append(ind)
        del ind,obj
        hourind = []
        hourextract = ERA5hour[int(np.min(np.asarray(dateind))):int(np.max(np.asarray(dateind)))+1]
        for ind,obj in enumerate(hourextract):
            if obj==hourtrack:            
                hourind.append(ind)
        allindices.append((int(np.min(np.asarray(dateind))),int(hourind[0])))
    return allindices

def extract_var(dataset=None,var='var138',indices=None):
    extractedvar = []
    for i in (range(len(indices))):
        realindex = indices[i][0]+indices[i][1]
        extractedvar.append(dataset[var][int(realindex),...].data)
    return np.asarray(extractedvar)

def smallarea(dataset=None,invar=None,indices=None,tc_irad=None):
    if len(invar.shape) != 3:
        invar = np.squeeze(invar)
    ds = xr.Dataset(
    data_vars=dict(variable=(["time","lat","lon"], invar)),#mysvar[0])),
    coords=dict(lat=(["lat"], dataset.lat.data),lon=(["lon"], dataset.lon.data),time=(["time"], np.linspace(0,len(indices)-1,len(indices)))),
    attrs=dict(description="coords with matrices"),)
    
    LATN,LATS,LONE,LONW = tc_irad[0,:]
    testsmall = ds['variable'][0,:,:].sel(lat=slice(LATS,LATN),lon=slice(LONE,LONW))
    if testsmall.shape[0]<testsmall.shape[1]:
        rgspt = int(testsmall.shape[0])
    else:
        rgspt = int(testsmall.shape[1])
    var_out=np.zeros((len(indices),rgspt,rgspt))
    del testsmall
    
    for it in range(len(indices)):
        latn, lats, lone, lonw = tc_irad[it,:]
        try:
            var_out[it,:,:]=ds['variable'][it,:,:].sel(lat=slice(lats,latn),lon=slice(lone,lonw))
        except:
            var_out[it,:,:]=ds['variable'][it,:,:].sel(lat=slice(lats,latn),lon=slice(lone,lonw))[0:rgspt,0:rgspt]
    return var_out
###############################read year from data and track ####################################

def readyear(year=None):
    dm2 = xr.open_dataset(datapath+'/slev_vars/svars_'+str(year)+'.nc')
    tracklist = sorted(glob.glob('/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/tracks_wp/*_'+str(year)+'*'))
    era5_date = [str(dm2.time[i].data).split('T')[0] for i in range(len(dm2.time))]
    era5_hour = [str(dm2.time[i].data).split('T')[1][0:2] for i in range(len(dm2.time))]
    return dm2,tracklist,era5_date,era5_hour

####change year below##########################
dm2,tracklist,era5_date,era5_hour = readyear(2011) 

TCsl_ts = []
for TCobj in tqdm(tracklist):
    track=pd.read_csv(TCobj,delimiter=r",")
    lon1=track['lon'].to_numpy()
    lat1=track['lat'].to_numpy()
    pos = arr = np.stack((lat1, lon1), axis=1)
    ###########################################################################
    indices_store = output_indices(track,era5_date,era5_hour)
    ###########################################################################
    mysvar = [extract_var(dataset=dm2,var=obj,indices=indices_store) for obj in (list(dm2.keys()))]
    ###########################################################################
    tc_irad=np.empty((len(indices_store),4))
    tc_irad[:,0] = pos[:,0]-2
    tc_irad[:,1] = pos[:,0]+2
    tc_irad[:,2] = pos[:,1]-2
    tc_irad[:,3] = pos[:,1]+2
    
    ################################################################################################
    
    smallsvarout = [smallarea(dm2,mysvar[i],indices_store,tc_irad) for i in (range(len(mysvar)))]
    
    svarname = ['u10','v10','2mdewtmp','2mtmp','conv_ppt','tot_cld_ice',\
           'tot_cldwtr','tot_cld_rain','vi_div_cld_froz_wtr','vi_div_cld_liq_wtr','vi_div_gpot_flux',\
           'vi_div_ke_flux','vi_div_mass_flux','vi_div_moisture_flux','vi_div_olr_flux','vi_div_tot_enrgy_flux',\
           'vi_ke','vi_pe_inte','vi_pe_ie_latentenrgy','vi_temp','vi_olr','vi_tot_enrgy','vi_moisture_div']
    svardict = {varnameobj:varobj for (varnameobj,varobj) in zip(svarname,smallsvarout)}
    
    ##################################################################################################
    #############################################################################################
    tsdict = {}
    for ind,obj in (enumerate(svarname)):
        tslist = [svardict[svarname[ind]][i,...].flatten() for i in range(len(indices_store))]
        tsdict[svarname[ind]] = [np.nanmean(obj) for obj in tslist]
    #############################################################################################
    TCsl_ts.append(tsdict)

  0%|          | 0/4 [00:00<?, ?it/s]

In [98]:
##########radiation variables############
def output_indices(TCtrack=None,ERA5date=None,ERA5hour=None):
    allindices = []
    for timeidx in range(len(TCtrack)):#len(track['time'])):
        datetrack,hourtrack = TCtrack['time'][timeidx].split(':')[0],TCtrack['time'][timeidx].split(':')[1][0:2]
        ####################################################################################################
        # Find the indices in ERA5 data with the same date as track
        ####################################################################################################
        dateind = []
        for ind,obj in enumerate(ERA5date):
            if obj==datetrack:
                dateind.append(ind)
        del ind,obj
        hourind = []
        hourextract = ERA5hour[int(np.min(np.asarray(dateind))):int(np.max(np.asarray(dateind)))+1]
        for ind,obj in enumerate(hourextract):
            if obj==hourtrack:            
                hourind.append(ind)
        allindices.append((int(np.min(np.asarray(dateind))),int(hourind[0])))
    return allindices

def extract_var(dataset=None,var='var138',indices=None):
    extractedvar = []
    for i in (range(len(indices))):
        realindex = indices[i][0]+indices[i][1]
        extractedvar.append(dataset[var][int(realindex),...].data)
    return np.asarray(extractedvar)

def smallarea(dataset=None,invar=None,indices=None,tc_irad=None):
    if len(invar.shape) != 3:
        invar = np.squeeze(invar)
    ds = xr.Dataset(
    data_vars=dict(variable=(["time","lat","lon"], invar)),#mysvar[0])),
    coords=dict(lat=(["lat"], dataset.lat.data),lon=(["lon"], dataset.lon.data),time=(["time"], np.linspace(0,len(indices)-1,len(indices)))),
    attrs=dict(description="coords with matrices"),)
    
    LATN,LATS,LONE,LONW = tc_irad[0,:]
    testsmall = ds['variable'][0,:,:].sel(lat=slice(LATS,LATN),lon=slice(LONE,LONW))
    if testsmall.shape[0]<testsmall.shape[1]:
        rgspt = int(testsmall.shape[0])
    else:
        rgspt = int(testsmall.shape[1])
    var_out=np.zeros((len(indices),rgspt,rgspt))
    del testsmall
    
    for it in range(len(indices)):
        latn, lats, lone, lonw = tc_irad[it,:]
        try:
            var_out[it,:,:]=ds['variable'][it,:,:].sel(lat=slice(lats,latn),lon=slice(lone,lonw))
        except:
            var_out[it,:,:]=ds['variable'][it,:,:].sel(lat=slice(lats,latn),lon=slice(lone,lonw))[0:rgspt,0:rgspt]
    return var_out
###############################read year from data and track ####################################

def readyear(year=None):
    dm2 = xr.open_dataset(datapath+'/slev_vars/radvars_'+str(year)+'.nc')
    tracklist = sorted(glob.glob('/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/tracks_wp/*_'+str(year)+'*'))
    era5_date = [str(dm2.time[i].data).split('T')[0] for i in range(len(dm2.time))]
    era5_hour = [str(dm2.time[i].data).split('T')[1][0:2] for i in range(len(dm2.time))]
    return dm2,tracklist,era5_date,era5_hour

dm2,tracklist,era5_date,era5_hour = readyear(2011)

TCrl_ts = []
for TCobj in tqdm(tracklist):
    track=pd.read_csv(TCobj,delimiter=r",")
    lon1=track['lon'].to_numpy()
    lat1=track['lat'].to_numpy()
    pos = arr = np.stack((lat1, lon1), axis=1)
    ###########################################################################
    indices_store = output_indices(track,era5_date,era5_hour)
    ###########################################################################
    mysvar = [extract_var(dataset=dm2,var=obj,indices=indices_store) for obj in (list(dm2.keys()))]
    ###########################################################################
    tc_irad=np.empty((len(indices_store),4))
    tc_irad[:,0] = pos[:,0]-2
    tc_irad[:,1] = pos[:,0]+2
    tc_irad[:,2] = pos[:,1]-2
    tc_irad[:,3] = pos[:,1]+2
    
    ################################################################################################
    
    smallsvarout = [smallarea(dm2,mysvar[i],indices_store,tc_irad) for i in (range(len(mysvar)))]
    
    svarname = ['cape','inst_10m_wnd_gst','inst_moisture_flux','inst_ssh_flux','surfmean_swr_flux','surfmean_lhf',\
           'surfmean_lwr_flux','surfmean_shf','dwnwrdmean_swr_flux','topmean_lwr_flux','topmean_swr_flux',\
           'vimean_moisture_div','surf_lhf','surf_shf','tot_suprcool_liqwtr','tot_wtr_vpr']
    
    svardict = {varnameobj:varobj for (varnameobj,varobj) in zip(svarname,smallsvarout)}
    
    ##################################################################################################
    #############################################################################################
    tsdict = {}
    for ind,obj in (enumerate(svarname)):
        tslist = [svardict[svarname[ind]][i,...].flatten() for i in range(len(indices_store))]
        tsdict[svarname[ind]] = [np.nanmean(obj) for obj in tslist]
    #############################################################################################
    TCrl_ts.append(tsdict)

  0%|          | 0/4 [00:00<?, ?it/s]

In [113]:
##########precip, mslp, sst variables############
def output_indices(TCtrack=None,ERA5date=None,ERA5hour=None):
    allindices = []
    for timeidx in range(len(TCtrack)):#len(track['time'])):
        datetrack,hourtrack = TCtrack['time'][timeidx].split(':')[0],TCtrack['time'][timeidx].split(':')[1][0:2]
        ####################################################################################################
        # Find the indices in ERA5 data with the same date as track
        ####################################################################################################
        dateind = []
        for ind,obj in enumerate(ERA5date):
            if obj==datetrack:
                dateind.append(ind)
        del ind,obj
        hourind = []
        hourextract = ERA5hour[int(np.min(np.asarray(dateind))):int(np.max(np.asarray(dateind)))+1]
        for ind,obj in enumerate(hourextract):
            if obj==hourtrack:            
                hourind.append(ind)
        allindices.append((int(np.min(np.asarray(dateind))),int(hourind[0])))
    return allindices

def extract_var(dataset=None,var='var138',indices=None):
    extractedvar = []
    for i in (range(len(indices))):
        realindex = indices[i][0]+indices[i][1]
        extractedvar.append(dataset[var][int(realindex),...].data)
    return np.asarray(extractedvar)

def smallarea(dataset=None,invar=None,indices=None,tc_irad=None):
    if len(invar.shape) != 3:
        invar = np.squeeze(invar)
    ds = xr.Dataset(
    data_vars=dict(variable=(["time","lat","lon"], invar)),#mysvar[0])),
    coords=dict(lat=(["lat"], dataset.lat.data),lon=(["lon"], dataset.lon.data),time=(["time"], np.linspace(0,len(indices)-1,len(indices)))),
    attrs=dict(description="coords with matrices"),)
    
    LATN,LATS,LONE,LONW = tc_irad[0,:]
    testsmall = ds['variable'][0,:,:].sel(lat=slice(LATS,LATN),lon=slice(LONE,LONW))
    if testsmall.shape[0]<testsmall.shape[1]:
        rgspt = int(testsmall.shape[0])
    else:
        rgspt = int(testsmall.shape[1])
    var_out=np.zeros((len(indices),rgspt,rgspt))
    del testsmall
    
    for it in range(len(indices)):
        latn, lats, lone, lonw = tc_irad[it,:]
        try:
            var_out[it,:,:]=ds['variable'][it,:,:].sel(lat=slice(lats,latn),lon=slice(lone,lonw))
        except:
            var_out[it,:,:]=ds['variable'][it,:,:].sel(lat=slice(lats,latn),lon=slice(lone,lonw))[0:rgspt,0:rgspt]
    return var_out
###############################read year from data and track ####################################

def readyear(year=None):
    dm2 = xr.open_mfdataset([datapath+'/prate/prates_2011.nc',datapath+'/mslp/mslp_2011.nc',datapath+'/sst/sst_2011.nc'])
    tracklist = sorted(glob.glob('/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/tracks_wp/*_'+str(year)+'*'))
    era5_date = [str(dm2.time[i].data).split('T')[0] for i in range(len(dm2.time))]
    era5_hour = [str(dm2.time[i].data).split('T')[1][0:2] for i in range(len(dm2.time))]
    return dm2,tracklist,era5_date,era5_hour

dm2,tracklist,era5_date,era5_hour = readyear(2011)

TCpr_ts = []
for TCobj in tqdm(tracklist):
    track=pd.read_csv(TCobj,delimiter=r",")
    lon1=track['lon'].to_numpy()
    lat1=track['lat'].to_numpy()
    pos = arr = np.stack((lat1, lon1), axis=1)
    ###########################################################################
    indices_store = output_indices(track,era5_date,era5_hour)
    ###########################################################################
    mysvar = [extract_var(dataset=dm2,var=obj,indices=indices_store) for obj in (list(dm2.keys()))]
    ###########################################################################
    tc_irad=np.empty((len(indices_store),4))
    tc_irad[:,0] = pos[:,0]-2
    tc_irad[:,1] = pos[:,0]+2
    tc_irad[:,2] = pos[:,1]-2
    tc_irad[:,3] = pos[:,1]+2
    
    ################################################################################################
    
    smallsvarout = [smallarea(dm2,mysvar[i],indices_store,tc_irad) for i in (range(len(mysvar)))]
    
    svarname = ['mslp','conv_rrate','ls_rrate','mn_conv_prate','mn_ls_prate','mn_tot_prate','sst']
    
    svardict = {varnameobj:varobj for (varnameobj,varobj) in zip(svarname,smallsvarout)}
    
    ##################################################################################################
    #############################################################################################
    tsdict = {}
    for ind,obj in (enumerate(svarname)):
        tslist = [svardict[svarname[ind]][i,...].flatten() for i in range(len(indices_store))]
        tsdict[svarname[ind]] = [np.nanmean(obj) for obj in tslist]
    #############################################################################################
    TCpr_ts.append(tsdict)

  0%|          | 0/4 [00:00<?, ?it/s]

/tmp/ipykernel_3307153/1148851969.py:94: RuntimeWarning: Mean of empty slice
  tsdict[svarname[ind]] = [np.nanmean(obj) for obj in tslist]


In [114]:
#For vars with 21 vertical levels vorticity humidity file##################
TCp_ts = []
for TCobj in tqdm(tracklist):
    track=pd.read_csv(TCobj,delimiter=r",")
    lon1=track['lon'].to_numpy()
    lat1=track['lat'].to_numpy()
    pos = arr = np.stack((lat1, lon1), axis=1)
    ###########################################################################
    indices_store = output_indices(track,era5_date,era5_hour)
    ###########################################################################
    mypvar = [extract_var(dataset=dm1,var=obj,indices=indices_store) for obj in (list(dm1.keys()))]
    ###########################################################################
    tc_irad=np.empty((len(indices_store),4))
    tc_irad[:,0] = pos[:,0]-2
    tc_irad[:,1] = pos[:,0]+2
    tc_irad[:,2] = pos[:,1]-2
    tc_irad[:,3] = pos[:,1]+2 
    
    ###########################################################################
    smallpvarout = [largearea_withpres(dm1,mypvar[i],indices_store,tc_irad) for i in (range(len(mypvar)))]    
    pvarname = ['vort','rhum']
    pvardict = {varnameobj:varobj for (varnameobj,varobj) in zip(pvarname,smallpvarout)}
    
    
    #############################################################################################
    ts_pdict = {}
    for ind,obj in (enumerate(pvarname)):
        pvarTS_store = []
        for plevv in range(len(dm1.plev.data)):
            tempvar = pvardict[pvarname[ind]][:,plevv,...]
            tempts = [tempvar[i,...].flatten() for i in range(len(indices_store))]
            tempTSERIES = [np.nanmean(obj) for obj in tempts]
            pvarTS_store.append(tempTSERIES)
        ts_pdict[pvarname[ind]] = np.asarray(pvarTS_store).transpose()
    #############################################################################################
    TCp_ts.append(ts_pdict)

  0%|          | 0/4 [00:00<?, ?it/s]

In [115]:
dm1=xr.open_dataset(datapath+'/gpot/gpot_2011.nc')

In [116]:
#geopotential
TCgp_ts = []
for TCobj in tqdm(tracklist):
    track=pd.read_csv(TCobj,delimiter=r",")
    lon1=track['lon'].to_numpy()
    lat1=track['lat'].to_numpy()
    pos = arr = np.stack((lat1, lon1), axis=1)
    ###########################################################################
    indices_store = output_indices(track,era5_date,era5_hour)
    ###########################################################################
    mypvar = [extract_var(dataset=dm1,var=obj,indices=indices_store) for obj in (list(dm1.keys()))]
    ###########################################################################
    tc_irad=np.empty((len(indices_store),4))
    tc_irad[:,0] = pos[:,0]-2
    tc_irad[:,1] = pos[:,0]+2
    tc_irad[:,2] = pos[:,1]-2
    tc_irad[:,3] = pos[:,1]+2 
    
    ###########################################################################
    smallpvarout = [largearea_withpres(dm1,mypvar[i],indices_store,tc_irad) for i in (range(len(mypvar)))]    
    pvarname = ['gpot']
    pvardict = {varnameobj:varobj for (varnameobj,varobj) in zip(pvarname,smallpvarout)}
    
    
    #############################################################################################
    ts_pdict = {}
    for ind,obj in (enumerate(pvarname)):
        pvarTS_store = []
        for plevv in range(len(dm1.plev.data)):
            tempvar = pvardict[pvarname[ind]][:,plevv,...]
            tempts = [tempvar[i,...].flatten() for i in range(len(indices_store))]
            tempTSERIES = [np.nanmean(obj) for obj in tempts]
            pvarTS_store.append(tempTSERIES)
        ts_pdict[pvarname[ind]] = np.asarray(pvarTS_store).transpose()
    #############################################################################################
    TCgp_ts.append(ts_pdict)

  0%|          | 0/4 [00:00<?, ?it/s]

In [117]:
dm1=xr.open_dataset(datapath+'/vvel/w_wnd_2011.nc')

In [118]:
#Vertical velocity
TCvv_ts = []
for TCobj in tqdm(tracklist):
    track=pd.read_csv(TCobj,delimiter=r",")
    lon1=track['lon'].to_numpy()
    lat1=track['lat'].to_numpy()
    pos = arr = np.stack((lat1, lon1), axis=1)
    ###########################################################################
    indices_store = output_indices(track,era5_date,era5_hour)
    ###########################################################################
    mypvar = [extract_var(dataset=dm1,var=obj,indices=indices_store) for obj in (list(dm1.keys()))]
    ###########################################################################
    tc_irad=np.empty((len(indices_store),4))
    tc_irad[:,0] = pos[:,0]-2
    tc_irad[:,1] = pos[:,0]+2
    tc_irad[:,2] = pos[:,1]-2
    tc_irad[:,3] = pos[:,1]+2 
    
    ###########################################################################
    smallpvarout = [largearea_withpres(dm1,mypvar[i],indices_store,tc_irad) for i in (range(len(mypvar)))]    
    pvarname = ['vvel']
    pvardict = {varnameobj:varobj for (varnameobj,varobj) in zip(pvarname,smallpvarout)}
    
    
    #############################################################################################
    ts_pdict = {}
    for ind,obj in (enumerate(pvarname)):
        pvarTS_store = []
        for plevv in range(len(dm1.plev.data)):
            tempvar = pvardict[pvarname[ind]][:,plevv,...]
            tempts = [tempvar[i,...].flatten() for i in range(len(indices_store))]
            tempTSERIES = [np.nanmean(obj) for obj in tempts]
            pvarTS_store.append(tempTSERIES)
        ts_pdict[pvarname[ind]] = np.asarray(pvarTS_store).transpose()
    #############################################################################################
    TCvv_ts.append(ts_pdict)

  0%|          | 0/4 [00:00<?, ?it/s]

In [119]:
tracklist

['/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/tracks_wp/nwp_20110519.csv',
 '/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/tracks_wp/nwp_20110616.csv',
 '/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/tracks_wp/nwp_20110926.csv',
 '/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/tracks_wp/nwp_20111211.csv']

In [123]:
preslv = [str(int(obj)) for obj in dm1.plev.data/100]

In [124]:
dx=xr.open_dataset(datapath+'/tlevs/tlevs_2011.nc')

In [122]:
plev=[str(int(obj)) for obj in dx.plev.data/100]

In [125]:
maindf_songda=pd.read_csv(target+'2011_w10max_nwp_songda.csv',delimiter=r",")
maindf_songda.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
maindf_songda=maindf_songda.drop('a', axis=1)

extra=pd.read_csv(output+'nwp/inder_2011_nwp_songda.csv',delimiter=r",")
extra.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
extra=extra.drop('a', axis=1)

rlev_songda=pd.DataFrame.from_dict(TCrl_ts[0])
prlev_songda=pd.DataFrame.from_dict(TCpr_ts[0])
slev_songda=pd.DataFrame.from_dict(TCsl_ts[0])
prlev_songda['mslp']=prlev_songda['mslp']/100

ts1_songda=pd.concat([maindf_songda,extra,slev_songda,rlev_songda,prlev_songda], axis=1, join='inner')

tempvortdict = {'vort_'+plev[i]:TCp_ts[0]['vort'][:,i] for i in range(21)}
temprhumdict = {'rhum_'+plev[i]:TCp_ts[0]['rhum'][:,i] for i in range(21)}
tempgpotdict = {'gpot_'+plev[i]:TCgp_ts[0]['gpot'][:,i] for i in range(21)}
tempvveldict = {'vvel_'+preslv[i]:TCvv_ts[0]['vvel'][:,i] for i in range(18)}
dict1 = {**tempvortdict,**temprhumdict}
dict3 = {**tempgpotdict,**tempvveldict}

alldict = {**dict1,**dict3}

plev_songda=pd.DataFrame.from_dict(alldict)
ts_songda=pd.concat([ts1_songda,plev_songda], axis=1, join='inner')
ts_songda

,w10max,div100,div200,div250,div300,div400,div500,eqt1000,eqt200,eqt250,...,vvel_250,vvel_300,vvel_400,vvel_500,vvel_600,vvel_700,vvel_850,vvel_925,vvel_975,vvel_1000
0,10.718504,0.000077,0.000052,2.137697e-05,0.000005,-0.000017,-0.000020,336.092651,350.741821,349.784058,...,-0.670697,-0.722765,-0.676007,-0.461145,-0.324112,-0.296068,-0.206768,-0.111188,-0.044621,-0.010296
1,11.079134,0.000071,0.000047,1.221514e-05,-0.000009,-0.000033,-0.000018,336.750488,350.423828,349.476318,...,-0.544893,-0.535496,-0.324848,-0.059601,0.015985,-0.071944,-0.113553,-0.064313,-0.019868,0.003074
2,11.393514,0.000065,0.000046,3.128390e-06,-0.000018,-0.000024,-0.000004,335.092499,350.678101,349.464355,...,-0.535979,-0.481105,-0.238823,-0.089775,-0.128695,-0.229063,-0.230868,-0.131252,-0.043195,-0.001989
3,10.368273,0.000058,0.000034,2.973929e-06,-0.000022,-0.000009,0.000010,335.330933,351.062988,349.878174,...,-0.458347,-0.396700,-0.211494,-0.221577,-0.363294,-0.447189,-0.305543,-0.165179,-0.066427,-0.020596
4,9.976253,0.000058,0.000047,2.414852e-06,-0.000023,-0.000002,0.000009,336.175537,351.966309,350.591736,...,-0.573811,-0.507423,-0.352300,-0.414862,-0.469124,-0.458226,-0.238681,-0.118327,-0.043880,-0.015749
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,31.007729,0.000043,0.000016,1.074035e-05,0.000011,0.000015,0.000020,327.064941,357.946259,356.405396,...,-0.224519,-0.286052,-0.350141,-0.574485,-0.706221,-0.813960,-0.747465,-0.472037,-0.179372,-0.105623
72,27.007444,0.000033,0.000009,1.288182e-05,0.000018,0.000014,0.000017,326.311798,357.456329,355.624268,...,-0.169809,-0.259317,-0.368785,-0.568877,-0.711004,-0.891589,-0.632799,-0.454616,-0.167393,-0.095902
73,26.172678,0.000025,0.000010,-7.005151e-07,0.000016,0.000006,0.000018,322.962097,356.528564,354.116028,...,-0.076199,-0.115631,-0.213639,-0.347600,-0.557422,-0.811936,-0.500391,-0.353326,-0.123278,-0.061714
74,24.936945,-0.000014,-0.000003,-2.516269e-06,0.000002,0.000021,0.000014,318.323547,356.221191,353.287964,...,0.042497,0.046916,-0.024819,-0.257797,-0.293782,-0.350950,-0.212255,-0.185276,-0.072762,-0.026945


In [126]:
ts_songda.to_csv(output+'inner/inner_wp_all_songda.csv')

In [127]:
maindf_haima=pd.read_csv(target+'2011_w10max_nwp_haima.csv',delimiter=r",")
maindf_haima.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
maindf_haima=maindf_haima.drop('a', axis=1)

extra=pd.read_csv(output+'nwp/inder_2011_nwp_haima.csv',delimiter=r",")
extra.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
extra=extra.drop('a', axis=1)

rlev_haima=pd.DataFrame.from_dict(TCrl_ts[1])
prlev_haima=pd.DataFrame.from_dict(TCpr_ts[1])
slev_haima=pd.DataFrame.from_dict(TCsl_ts[1])
prlev_haima['mslp']=prlev_haima['mslp']/100

ts1_haima=pd.concat([maindf_haima,extra,slev_haima,rlev_haima,prlev_haima], axis=1, join='inner')

tempvortdict = {'vort_'+plev[i]:TCp_ts[1]['vort'][:,i] for i in range(21)}
temprhumdict = {'rhum_'+plev[i]:TCp_ts[1]['rhum'][:,i] for i in range(21)}
tempgpotdict = {'gpot_'+plev[i]:TCgp_ts[1]['gpot'][:,i] for i in range(21)}
tempvveldict = {'vvel_'+preslv[i]:TCvv_ts[1]['vvel'][:,i] for i in range(18)}
dict1 = {**tempvortdict,**temprhumdict}
dict3 = {**tempgpotdict,**tempvveldict}

alldict = {**dict1,**dict3}

plev_haima=pd.DataFrame.from_dict(alldict)
ts_haima=pd.concat([ts1_haima,plev_haima], axis=1, join='inner')
ts_haima

,w10max,div100,div200,div250,div300,div400,div500,eqt1000,eqt200,eqt250,...,vvel_250,vvel_300,vvel_400,vvel_500,vvel_600,vvel_700,vvel_850,vvel_925,vvel_975,vvel_1000
0,12.920896,0.000031,0.000026,0.000012,0.000005,0.000005,-1.391179e-06,332.352936,350.438232,349.183289,...,-0.369178,-0.407269,-0.432053,-0.469188,-0.394138,-0.266210,-0.179414,-0.110342,-0.060262,-0.033859
1,11.155981,0.000024,0.000034,0.000013,0.000004,0.000003,-8.728022e-06,334.635468,350.661316,349.683777,...,-0.360079,-0.390059,-0.447146,-0.400113,-0.324511,-0.207066,-0.130560,-0.082442,-0.027006,0.003606
2,11.616890,0.000027,0.000020,0.000017,0.000007,-0.000006,-1.252973e-06,335.600983,350.385376,349.080017,...,-0.283832,-0.338717,-0.376513,-0.278730,-0.299669,-0.226455,-0.130422,-0.097112,-0.043724,-0.011371
3,10.947051,0.000032,0.000002,0.000011,0.000005,-0.000007,2.637102e-06,336.121765,350.589172,348.551758,...,-0.181734,-0.230602,-0.219748,-0.179790,-0.233788,-0.233020,-0.154049,-0.119693,-0.050673,-0.011326
4,12.182612,0.000046,-0.000007,-0.000002,0.000001,-0.000003,-2.026347e-07,335.806244,350.569824,348.603302,...,-0.164798,-0.171781,-0.138739,-0.115175,-0.142810,-0.219130,-0.233484,-0.176286,-0.093232,-0.046416
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,11.428171,0.000028,0.000025,0.000015,0.000011,0.000008,-2.628424e-07,343.384369,358.291931,356.843018,...,-0.412611,-0.474131,-0.575956,-0.622354,-0.606087,-0.566192,-0.463619,-0.200473,-0.076915,-0.069815
73,11.366698,0.000031,0.000024,0.000018,0.000009,0.000004,-1.728417e-06,341.531067,358.682953,357.107849,...,-0.426512,-0.494524,-0.570164,-0.585540,-0.530935,-0.530132,-0.370454,-0.148072,-0.059226,-0.056570
74,9.420424,0.000034,0.000030,0.000008,0.000005,0.000014,4.283335e-06,341.135956,357.978088,356.886780,...,-0.438158,-0.465204,-0.549787,-0.659518,-0.666009,-0.600372,-0.310339,-0.094681,-0.020869,-0.018876
75,9.506091,0.000036,0.000033,0.000014,0.000017,0.000011,-1.208833e-06,339.284729,357.697754,356.691986,...,-0.459210,-0.536060,-0.671587,-0.713178,-0.688363,-0.601546,-0.252709,-0.054258,0.000327,0.002019


In [128]:
ts_haima.to_csv(output+'inner/inner_wp_all_2011_haima.csv')

In [129]:
maindf_nalgae=pd.read_csv(target+'2011_w10max_nwp_nalgae.csv',delimiter=r",")
maindf_nalgae.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
maindf_nalgae=maindf_nalgae.drop('a', axis=1)

extra=pd.read_csv(output+'nwp/inder_2011_nwp_nalgae.csv',delimiter=r",")
extra.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
extra=extra.drop('a', axis=1)

rlev_nalgae=pd.DataFrame.from_dict(TCrl_ts[2])
prlev_nalgae=pd.DataFrame.from_dict(TCpr_ts[2])
slev_nalgae=pd.DataFrame.from_dict(TCsl_ts[2])
prlev_nalgae['mslp']=prlev_nalgae['mslp']/100

ts1_nalgae=pd.concat([maindf_nalgae,extra,slev_nalgae,rlev_nalgae,prlev_nalgae], axis=1, join='inner')

tempvortdict = {'vort_'+plev[i]:TCp_ts[2]['vort'][:,i] for i in range(21)}
temprhumdict = {'rhum_'+plev[i]:TCp_ts[2]['rhum'][:,i] for i in range(21)}
tempgpotdict = {'gpot_'+plev[i]:TCgp_ts[2]['gpot'][:,i] for i in range(21)}
tempvveldict = {'vvel_'+preslv[i]:TCvv_ts[2]['vvel'][:,i] for i in range(18)}
dict1 = {**tempvortdict,**temprhumdict}
dict3 = {**tempgpotdict,**tempvveldict}

alldict = {**dict1,**dict3}

plev_nalgae=pd.DataFrame.from_dict(alldict)
ts_nalgae=pd.concat([ts1_nalgae,plev_nalgae], axis=1, join='inner')
ts_nalgae

,w10max,div100,div200,div250,div300,div400,div500,eqt1000,eqt200,eqt250,...,vvel_250,vvel_300,vvel_400,vvel_500,vvel_600,vvel_700,vvel_850,vvel_925,vvel_975,vvel_1000
0,14.322814,6.940629e-05,3.870234e-05,-4.192808e-06,-2.194361e-05,-1.767585e-05,-1.008410e-05,335.620361,351.562347,350.050415,...,-0.305694,-0.366121,-0.425241,-0.442577,-0.460280,-0.446590,-0.268834,-0.139309,-0.052781,-0.012822
1,12.952862,5.659458e-05,3.064299e-05,-8.663010e-07,-5.622924e-06,-1.080651e-05,-8.188302e-06,336.041443,351.699280,350.131805,...,-0.540881,-0.626316,-0.705279,-0.666350,-0.636788,-0.586059,-0.338140,-0.164387,-0.057875,-0.009832
2,11.460491,4.792262e-05,2.876717e-05,6.633320e-06,4.266361e-06,-6.333008e-06,-1.045010e-05,336.822693,351.721069,350.217072,...,-0.633640,-0.648437,-0.637554,-0.580432,-0.542309,-0.530566,-0.363803,-0.188739,-0.074721,-0.023225
3,11.687129,5.025869e-05,3.547810e-05,1.158262e-05,5.285798e-06,-7.670015e-06,-1.092796e-05,336.804626,351.301819,350.431396,...,-0.684348,-0.625032,-0.427362,-0.282664,-0.172797,-0.116783,-0.107429,-0.054103,-0.010569,0.009448
4,14.077259,7.611053e-05,3.487488e-05,-6.048032e-06,-4.199692e-07,1.948070e-06,-9.135704e-06,336.892578,352.463776,350.178406,...,-0.427495,-0.337061,-0.133257,-0.023278,0.009895,-0.006391,-0.090638,-0.084483,-0.047509,-0.023628
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,15.985020,-4.544082e-06,-6.333099e-06,5.510799e-06,2.582844e-06,5.426166e-06,-4.337699e-08,311.929688,350.406128,348.653687,...,-0.068294,-0.068211,-0.031154,-0.053859,-0.061257,-0.093983,-0.227037,-0.198510,-0.076411,-0.005234
69,14.828885,6.460446e-06,-3.414246e-06,1.175802e-05,-4.896817e-06,1.916140e-06,1.696170e-06,312.387817,350.122833,348.299225,...,0.000680,-0.017662,-0.000116,-0.004074,-0.041699,-0.130337,-0.261529,-0.234982,-0.107389,-0.034995
70,13.743028,2.002344e-06,5.525781e-07,1.222747e-05,-5.748053e-06,2.975128e-06,9.103449e-07,312.428162,350.013916,347.809021,...,0.031141,-0.016618,-0.038170,-0.070783,-0.108380,-0.202504,-0.300153,-0.254576,-0.115655,-0.042799
71,13.305566,-5.314167e-07,5.844769e-06,8.193862e-06,-4.943665e-06,-2.566430e-07,1.341856e-06,309.987518,349.912048,347.305786,...,0.029899,-0.005405,-0.035882,-0.062291,-0.083592,-0.181876,-0.277616,-0.208589,-0.071308,-0.004314


In [130]:
ts_nalgae.to_csv(output+'inner/inner_wp_all_nalgae.csv')

In [131]:
maindf_washi=pd.read_csv(target+'2011_w10max_nwp_washi.csv',delimiter=r",")
maindf_washi.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
maindf_washi=maindf_washi.drop('a', axis=1)

extra=pd.read_csv(output+'nwp/inder_2011_nwp_washi.csv',delimiter=r",")
extra.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
extra=extra.drop('a', axis=1)

rlev_washi=pd.DataFrame.from_dict(TCrl_ts[3])
prlev_washi=pd.DataFrame.from_dict(TCpr_ts[3])
slev_washi=pd.DataFrame.from_dict(TCsl_ts[3])
prlev_washi['mslp']=prlev_washi['mslp']/100

ts1_washi=pd.concat([maindf_washi,extra,slev_washi,rlev_washi,prlev_washi], axis=1, join='inner')

tempvortdict = {'vort_'+plev[i]:TCp_ts[3]['vort'][:,i] for i in range(21)}
temprhumdict = {'rhum_'+plev[i]:TCp_ts[3]['rhum'][:,i] for i in range(21)}
tempgpotdict = {'gpot_'+plev[i]:TCgp_ts[3]['gpot'][:,i] for i in range(21)}
tempvveldict = {'vvel_'+preslv[i]:TCvv_ts[3]['vvel'][:,i] for i in range(18)}
dict1 = {**tempvortdict,**temprhumdict}
dict3 = {**tempgpotdict,**tempvveldict}

alldict = {**dict1,**dict3}

plev_washi=pd.DataFrame.from_dict(alldict)
ts_washi=pd.concat([ts1_washi,plev_washi], axis=1, join='inner')
ts_washi

,w10max,div100,div200,div250,div300,div400,div500,eqt1000,eqt200,eqt250,...,vvel_250,vvel_300,vvel_400,vvel_500,vvel_600,vvel_700,vvel_850,vvel_925,vvel_975,vvel_1000
0,11.374318,-0.000024,6.228312e-06,2.071853e-05,0.000008,-9.523897e-06,-2.231343e-07,314.772888,350.313934,347.112366,...,-0.113661,-0.142110,-0.181867,-0.213029,-0.152688,-0.077119,-0.004886,0.015413,0.006475,-0.001271
1,12.570400,-0.000014,2.781810e-06,1.150798e-05,0.000003,-8.442781e-06,6.725560e-06,313.111267,350.093811,347.030518,...,0.009026,-0.019765,-0.138412,-0.192658,-0.111991,-0.034952,-0.011498,0.018398,0.018842,0.018221
2,12.993572,-0.000005,-2.801417e-07,-2.842421e-06,-0.000002,-3.891363e-06,8.958064e-06,313.254547,350.107147,347.239014,...,-0.137606,-0.198861,-0.243741,-0.228448,-0.137544,-0.067224,-0.050411,-0.018866,-0.010759,-0.013463
3,12.251040,-0.000002,1.103165e-05,-8.869802e-08,0.000004,1.414512e-06,6.908057e-06,317.138824,349.477478,347.601807,...,-0.089987,-0.117206,-0.107977,-0.025911,0.069142,0.069723,-0.004144,0.004833,-0.008240,-0.021445
4,12.024080,-0.000011,9.125998e-06,3.205580e-06,0.000008,7.640495e-07,6.252454e-06,317.027039,349.305023,348.102783,...,-0.066571,-0.099347,-0.124154,-0.021825,0.066585,0.060728,0.004286,0.018940,0.024510,0.022577
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,18.803926,0.000039,1.170732e-05,-2.679585e-07,-0.000002,-7.172116e-07,-4.642721e-06,335.051941,353.788757,350.431610,...,-0.286011,-0.257260,-0.147831,-0.208916,-0.312755,-0.430523,-0.436728,-0.286488,-0.134242,-0.043402
57,18.780231,0.000031,-3.424115e-06,-1.704585e-05,-0.000010,7.581883e-06,-1.079770e-06,334.113342,353.285675,350.282227,...,-0.305626,-0.329236,-0.318661,-0.416843,-0.535565,-0.600969,-0.501054,-0.311903,-0.120404,-0.011977
58,18.004267,0.000015,-5.888951e-06,-1.915601e-05,-0.000010,7.138413e-06,6.689090e-07,331.277893,352.275787,350.156281,...,-0.283775,-0.283752,-0.307421,-0.407677,-0.444012,-0.462828,-0.376844,-0.250800,-0.095736,-0.007259
59,16.774679,0.000011,-1.164030e-05,-1.414149e-05,-0.000006,2.098375e-06,-2.144409e-06,334.114502,352.113495,349.772400,...,-0.283951,-0.270899,-0.322254,-0.407174,-0.383868,-0.410295,-0.390141,-0.278266,-0.122072,-0.037444


In [132]:
ts_washi.to_csv(output+'inner/inner_wp_all_washi.csv')

In [83]:
maindf_bopha=pd.read_csv(target+'2012_w10max_nwp_bopha.csv',delimiter=r",")
maindf_bopha.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
maindf_bopha=maindf_bopha.drop('a', axis=1)

extra=pd.read_csv(output+'nwp/inder_2012_nwp_bopha.csv',delimiter=r",")
extra.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
extra=extra.drop('a', axis=1)

rlev_bopha=pd.DataFrame.from_dict(TCrl_ts[4])
prlev_bopha=pd.DataFrame.from_dict(TCpr_ts[4])
slev_bopha=pd.DataFrame.from_dict(TCsl_ts[4])
prlev_bopha['mslp']=prlev_bopha['mslp']/100

ts1_bopha=pd.concat([maindf_bopha,extra,slev_bopha,rlev_bopha,prlev_bopha], axis=1, join='inner')

tempvortdict = {'vort_'+plev[i]:TCp_ts[4]['vort'][:,i] for i in range(21)}
temprhumdict = {'rhum_'+plev[i]:TCp_ts[4]['rhum'][:,i] for i in range(21)}
tempgpotdict = {'gpot_'+plev[i]:TCgp_ts[4]['gpot'][:,i] for i in range(21)}
tempvveldict = {'vvel_'+preslv[i]:TCvv_ts[4]['vvel'][:,i] for i in range(18)}
dict1 = {**tempvortdict,**temprhumdict}
dict3 = {**tempgpotdict,**tempvveldict}

alldict = {**dict1,**dict3}

plev_bopha=pd.DataFrame.from_dict(alldict)
ts_bopha=pd.concat([ts1_bopha,plev_bopha], axis=1, join='inner')
ts_bopha

,w10max,div100,div200,div250,div300,div400,div500,eqt1000,eqt200,eqt250,...,vvel_250,vvel_300,vvel_400,vvel_500,vvel_600,vvel_700,vvel_850,vvel_925,vvel_975,vvel_1000
0,11.092641,0.000064,0.000042,0.000023,3.910915e-06,-4.563233e-06,-1.868183e-05,336.073883,350.519592,395.556580,...,-0.615487,-0.677856,-0.651869,-0.536190,-0.364659,-0.303984,-0.196733,-0.125479,-0.063901,-0.031237
1,9.667654,0.000048,0.000041,0.000025,1.155771e-05,-3.857489e-06,-2.412654e-05,336.479614,349.877228,394.340027,...,-0.476637,-0.563685,-0.622700,-0.446390,-0.221514,-0.138309,-0.074206,-0.063294,-0.044526,-0.030694
2,9.848610,0.000027,0.000032,0.000026,2.015611e-05,-4.935975e-06,-2.737122e-05,336.045593,349.157166,394.138641,...,-0.310796,-0.427736,-0.557608,-0.327135,-0.109885,-0.036088,0.022335,0.003958,0.001427,0.006403
3,9.333220,0.000014,0.000021,0.000030,2.707488e-05,-2.749592e-06,-1.301638e-05,335.500244,349.245117,394.340576,...,-0.150551,-0.296595,-0.465544,-0.319724,-0.181960,-0.144298,-0.070544,-0.034035,0.001526,0.018603
4,9.189720,0.000013,0.000017,0.000026,1.576553e-05,4.539037e-06,-1.087956e-05,335.900574,350.834473,395.941589,...,-0.190416,-0.295073,-0.384726,-0.364248,-0.318239,-0.316380,-0.207763,-0.126740,-0.062568,-0.030159
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,14.970678,0.000015,-0.000006,-0.000001,8.178898e-07,-1.223705e-05,1.842626e-06,331.840210,348.687927,395.573303,...,-0.020207,-0.018247,0.078498,0.074256,-0.066280,-0.241047,-0.330103,-0.215278,-0.084641,-0.015502
107,14.750920,0.000010,-0.000006,-0.000005,7.828398e-06,-6.495432e-06,-1.132198e-06,332.813812,348.504089,395.057922,...,0.016788,0.011981,0.022422,0.032425,-0.044260,-0.235432,-0.342298,-0.223966,-0.080462,0.003428
108,15.959667,0.000010,-0.000003,-0.000004,6.613144e-06,-1.746871e-06,-8.347175e-07,331.304413,348.503937,395.298828,...,0.007844,0.005218,-0.012099,-0.007879,-0.037072,-0.168013,-0.295899,-0.229792,-0.091670,-0.004617
109,15.455827,0.000004,-0.000003,-0.000003,1.294745e-07,-5.299962e-07,6.249441e-06,332.513489,348.450134,395.200745,...,0.030606,0.048182,0.028919,-0.001558,-0.081337,-0.219372,-0.340716,-0.260778,-0.148129,-0.071304


In [84]:
ts_bopha.to_csv(output+'inner/inner_wp_all_bopha.csv')

In [55]:
maindf_haiyan=pd.read_csv(target+'2013_w10max_nwp_haiyan.csv',delimiter=r",")
maindf_haiyan.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
maindf_haiyan=maindf_haiyan.drop('a', axis=1)

extra=pd.read_csv(output+'nwp/inder_2013_nwp_haiyan.csv',delimiter=r",")
extra.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
extra=extra.drop('a', axis=1)

rlev_haiyan=pd.DataFrame.from_dict(TCrl_ts[5])
prlev_haiyan=pd.DataFrame.from_dict(TCpr_ts[5])
slev_haiyan=pd.DataFrame.from_dict(TCsl_ts[5])
prlev_haiyan['mslp']=prlev_haiyan['mslp']/100

ts1_haiyan=pd.concat([maindf_haiyan,extra,slev_haiyan,rlev_haiyan,prlev_haiyan], axis=1, join='inner')

tempvortdict = {'vort_'+plev[i]:TCp_ts[5]['vort'][:,i] for i in range(21)}
temprhumdict = {'rhum_'+plev[i]:TCp_ts[5]['rhum'][:,i] for i in range(21)}
tempgpotdict = {'gpot_'+plev[i]:TCgp_ts[5]['gpot'][:,i] for i in range(21)}
tempvveldict = {'vvel_'+preslv[i]:TCvv_ts[5]['vvel'][:,i] for i in range(18)}
dict1 = {**tempvortdict,**temprhumdict}
dict3 = {**tempgpotdict,**tempvveldict}

alldict = {**dict1,**dict3}

plev_haiyan=pd.DataFrame.from_dict(alldict)
ts_haiyan=pd.concat([ts1_haiyan,plev_haiyan], axis=1, join='inner')
ts_haiyan

,w10max,div100,div200,div250,div300,div400,div500,eqt1000,eqt200,eqt250,...,vvel_250,vvel_300,vvel_400,vvel_500,vvel_600,vvel_700,vvel_850,vvel_925,vvel_975,vvel_1000
0,8.531227,0.000001,0.000023,1.902287e-05,0.000006,-0.000020,0.000003,328.512970,348.270355,347.171478,...,-0.141755,-0.212832,-0.032366,0.018735,-0.043674,-0.086838,-0.043365,-0.013033,0.003268,0.012539
1,8.789506,-0.000005,0.000022,1.894513e-05,0.000012,-0.000015,0.000003,326.717407,347.581848,346.546661,...,-0.114033,-0.196498,-0.106204,-0.080884,-0.103923,-0.122139,-0.048403,-0.010106,0.001106,0.009789
2,9.109775,0.000009,0.000026,2.287460e-05,0.000014,-0.000004,0.000008,332.980713,348.626038,347.415466,...,-0.241873,-0.336453,-0.342675,-0.404546,-0.430571,-0.397095,-0.211560,-0.122145,-0.059279,-0.027113
3,8.735093,0.000013,0.000024,2.707223e-05,0.000014,-0.000006,0.000002,329.524536,349.692078,347.838745,...,-0.306566,-0.407457,-0.392387,-0.394806,-0.370379,-0.321276,-0.166939,-0.104087,-0.053154,-0.027038
4,8.762020,0.000021,0.000021,1.660212e-05,0.000006,-0.000002,0.000004,329.931396,349.661682,348.227905,...,-0.253489,-0.314271,-0.275665,-0.308380,-0.313969,-0.282644,-0.141788,-0.081978,-0.030562,-0.002098
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,18.393751,0.000027,0.000055,4.400695e-05,0.000029,0.000012,0.000009,324.850250,353.000488,352.627747,...,-0.496628,-0.670299,-0.910785,-1.016337,-1.135077,-1.184386,-1.073887,-0.539167,-0.110862,-0.000668
71,16.716141,-0.000003,0.000045,2.842371e-05,0.000028,0.000045,0.000027,319.582214,353.327454,352.416473,...,-0.293517,-0.414011,-0.803800,-1.242996,-1.415408,-1.422070,-1.157575,-0.540096,-0.130186,-0.028915
72,13.369188,-0.000004,0.000025,1.420011e-05,0.000014,0.000049,0.000032,315.080811,353.871429,352.219818,...,-0.203652,-0.261752,-0.606687,-1.082970,-1.319246,-1.368640,-1.129714,-0.504243,-0.086246,0.012081
73,11.980754,0.000005,0.000010,9.524666e-06,0.000014,0.000040,0.000030,314.847198,353.513214,351.351532,...,-0.139279,-0.196142,-0.509415,-0.894445,-1.124738,-1.226175,-1.029234,-0.438954,-0.048670,0.037450


In [56]:
ts_haiyan.to_csv(output+'inner/inner_wp_all_haiyan.csv')